<a href="https://colab.research.google.com/github/Azorios/DL1/blob/main/prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prediction Notebook

Install necessary dependencies.

In [1]:
!pip install scikit-learn datasets
!pip install torch --index-url https://download.pytorch.org/whl/cu117 # for cuda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.4 MB/s eta 0:00:00
Looking in indexes: https://download.pytorch.org/whl/cu117, https://us-python.pkg.dev/colab-wheels/public/simple/


#Predict Probabilities and Imagelabels

Make sure to define myBatch = [yourArray of Images]

In [110]:
import torch
import numpy as np
from PIL import Image
from torchvision.transforms import Normalize, Compose, RandomResizedCrop, RandomHorizontalFlip, RandomVerticalFlip, ToTensor, Resize, CenterCrop, GaussianBlur, RandomApply

# Please add you array of [batchsize,1]
img=Image.open('./rub.jpg')
img2= Image.open('./aiKunst.png')
myBatch = [img,img2]


imgs = [img]
def predict(images):
  results = []
  for img in images: 
    output= torch.sigmoid(model(torch.tensor(test_transforms(img)).unsqueeze(0).to('cuda')))
    preds = torch.round(output)
    results.append([1-preds,preds])
  predictions = torch.tensor(results)
  return predictions

class myModel():
    test_transforms = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])
    def __init__(self):  
        self.model = torch.jit.load('./model_data/resnet50_model3.pth')
        self.model.eval()     
        
      
    def predict(self,images):      
      results = []
      predictions = []
      probabilities = []
      for img in images: 
        output= torch.sigmoid(self.model(torch.tensor(test_transforms(img)).unsqueeze(0).to('cuda')))
        preds = torch.round(output)
        probabilities.append(output)
        predictions.append(preds)        
      result = torch.tensor([probabilities, predictions]).T
      return result


model = myModel()

"""
The function takes in an array of images (myBatch) and returns a Tensor with the  probabilities of each Image being fake as well as the prediction of each image being fake. 

"""
prediction =model.predict(myBatch)

prediction


<ipython-input-110-7838ca8f3904>:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output= torch.sigmoid(self.model(torch.tensor(test_transforms(img)).unsqueeze(0).to('cuda')))


tensor([[0.4057, 0.0000],
        [0.6885, 1.0000]])

Import all needed packages.

In [2]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from datasets import load_dataset, concatenate_datasets
from torchvision.transforms import Normalize, Compose, Resize, ToTensor
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

Download dataset, remove unnecessary columns and add label 0 to real images and 1 to generated images.
Then split the dataset into a train (80%), validation (10%) and test dataset (10%).

In [3]:
def get_dataset():
    # datasets
    fake = load_dataset('poloclub/diffusiondb', '2m_random_10k', split='train', data_dir='./')
    real = load_dataset('frgfm/imagenette', '320px', split='train+validation', data_dir='./')

    # remove unnecessary columns
    fake = fake.remove_columns(
        ['prompt', 'seed', 'step', 'cfg', 'sampler', 'width', 'height', 'user_name', 'timestamp', 'image_nsfw',
         'prompt_nsfw'])
    real = real.remove_columns('label')

    # add label column with 0 for real images and for 1 for generated images
    fake = fake.map(lambda x: {'image': x['image'], 'label': 1})
    real = real.map(lambda x: {'image': x['image'], 'label': 0})

    # split fake dataset into train, validation and test sets
    fake_train_testvalid = fake.train_test_split(test_size=0.2)
    fake_test_valid = fake_train_testvalid['test'].train_test_split(test_size=0.5)

    # split real dataset into train, validation and test sets
    real_train_testvalid = real.train_test_split(test_size=0.2)
    real_test_valid = real_train_testvalid['test'].train_test_split(test_size=0.5)

    # combine fake and real datasets into single dataset for each split
    train_dataset = concatenate_datasets([fake_train_testvalid['train'], real_train_testvalid['train']])
    val_dataset = concatenate_datasets([fake_test_valid['train'], real_test_valid['train']])
    test_dataset = concatenate_datasets([fake_test_valid['test'], real_test_valid['test']])

    return train_dataset, val_dataset, test_dataset

Transform and preprocess test data. Convert the images to pixel_values (tensors).

In [4]:
test_transforms = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

def transform_test(batch):
    batch['pixel_values'] = [test_transforms(img.convert("RGB")) for img in batch['image']]
    del batch['image']
    return batch

Functions for prediction.

In [5]:
def get_image_label(data, device):
    """
    Get image and label data. Convert label data to the same nn output shape.
    """
    image = data['pixel_values'].to(device)
    label = data['label'].unsqueeze(1).float().to(device)

    return image, label


def predict_test(test_loader, model, device):
    """
    Predict all the test dataset and calculate confusion matrix, AUROC and classification report with f1 score, precision and recall.
    """
    predictions, targets = [], []

    model.eval()
    for data in test_loader:
        image, label = get_image_label(data, device)

        output = torch.sigmoid(model(image))
        preds = torch.round(output)

        preds = preds.detach().cpu().numpy()
        label = label.detach().cpu().numpy()

        for i in range(len(preds)):
            predictions.append(preds[i])
            targets.append((label[i]))

    predictions = np.array(predictions)
    targets = np.array(targets)

    conf_mat = confusion_matrix(targets, predictions)
    class_rep = classification_report(targets, predictions, target_names=('Real', 'Generated'))
    auroc = roc_auc_score(targets, predictions)

    print(conf_mat)
    print(class_rep)
    print(auroc)

    file = open(f'./model_data/resnet50_metrics.txt', 'w')
    file.write(f'ResNet50 Dataset1\n\nConfusion Matrix\n\n{conf_mat}\n\nClassification Report\n\n{class_rep}\n\n'
               f'AUROC: {auroc}')
    file.close()

Execute Prediction.

In [6]:
# used device for computing
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device, 'will be used.')

# get the dataset
_, _, test_dataset = get_dataset()

# apply transforms and preprocessing to dataset
test_dataset.set_transform(transform_test)

# load data
batch_size = 32
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=0)



cuda will be used.


Generating train split: 0 examples [00:00, ? examples/s]

Dataset diffusiondb downloaded and prepared to /root/.cache/huggingface/datasets/poloclub___diffusiondb/2m_random_10k-data_dir=.%2F/0.9.1/b3bc1e64570dc7149af62c4bac49ecfbce16b683dd4fee083292fae1afa95f7c. Subsequent calls will reuse this data.


Generating train split:   0%|          | 0/9469 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3925 [00:00<?, ? examples/s]

Dataset imagenette downloaded and prepared to /root/.cache/huggingface/datasets/frgfm___imagenette/320px-data_dir=.%2F/1.0.0/38929285b8abcae5c1305418e9d8fea5dd6b189bbbd22caba5f5537c7fa0f01f. Subsequent calls will reuse this data.


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/13394 [00:00<?, ? examples/s]